<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [0]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [0]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [7]:
#ANSWER:
response.headers['Content-Type']

'application/json'

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [0]:
#ANSWER:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [0]:
#ANSWER:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [10]:
#ANSWER:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Sat, 28 Mar 2020 00:52:45 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [11]:
#ANSWER:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1585356765, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 662, \n      "risetime": 1585358109\n    }, \n    {\n      "duration": 461, \n      "risetime": 1585363988\n    }, \n    {\n      "duration": 597, \n      "risetime": 1585412374\n    }, \n    {\n      "duration": 648, \n      "risetime": 1585418146\n    }, \n    {\n      "duration": 483, \n      "risetime": 1585424088\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [12]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [13]:
#ANSWER:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [14]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1585356765, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 662, 'risetime': 1585358109}, {'duration': 461, 'risetime': 1585363988}, {'duration': 597, 'risetime': 1585412374}, {'duration': 648, 'risetime': 1585418146}, {'duration': 483, 'risetime': 1585424088}]}


What kind of object did this give us?

In [15]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate dict:

In [0]:
#ANSWER:
passes = overheads['response']

In [20]:
passes[1]['risetime']

1585363988

In [23]:
x = []
for item in passes:
    x.append(item['risetime'])
x

[1585358109, 1585363988, 1585412374, 1585418146, 1585424088]

Now extract the `risetime` strings into an array called `srisetimes`:

In [24]:
#ANSWER:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1585358109, 1585363988, 1585412374, 1585418146, 1585424088]

It is Unix Timestamp. We wonvert these to an array of Python `datetime` values called `risetimes`:

In [0]:
#ANSWER:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

In [27]:
risetimes

[datetime.datetime(2020, 3, 28, 1, 15, 9),
 datetime.datetime(2020, 3, 28, 2, 53, 8),
 datetime.datetime(2020, 3, 28, 16, 19, 34),
 datetime.datetime(2020, 3, 28, 17, 55, 46),
 datetime.datetime(2020, 3, 28, 19, 34, 48)]

Finally, print these in a format that people understand:

In [28]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

28/03/20 01:15
28/03/20 02:53
28/03/20 04:19
28/03/20 05:55
28/03/20 07:34


Here is an endpoint that tells us who is onboard:

In [0]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [30]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'people': [{'craft': 'ISS', 'name': 'Andrew Morgan'}, {'craft': 'ISS', 'name': 'Oleg Skripochka'}, {'craft': 'ISS', 'name': 'Jessica Meir'}], 'message': 'success', 'number': 3}
3
Andrew Morgan
Oleg Skripochka
Jessica Meir


[None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [0]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [0]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

>
>
>
>
>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



